In [3]:
import cv2
import numpy as np
import torch
from torch.autograd import Variable
from torchvision import models
from __future__ import print_function, division 
import os 
import torch 
import pandas as pd 
from PIL import Image 
import numpy as np 
from torch.utils.data import Dataset, DataLoader 
from torchvision import transforms 
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import argparse
import time
#创建30个文件夹

In [2]:
def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')#将图片转换为RGB格式。

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor()])

In [ ]:
class FeatureVisualization():
    def __init__(self,model,img_path,transform,selected_layer,select_filter):
        self.img_path=img_path
        self.selected_layer=selected_layer
        self.pretrained_model = model
        self.transform=transform
    def imread(path):
        # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
        with open(path, 'rb') as f:
            img = Image.open(f)
        return img.convert('RGB')#将图片转换为RGB格式。
        #print( self.pretrained_model)
    def process_image(self):
        img=imread(self.img_path)
        img=self.transform(img)
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        if(torch.cuda.is_available()):
            img=img.to(device)
        return img

    def get_feature(self):
        # input = Variable(torch.randn(1, 3, 224, 224))
        input=self.process_image()
        #print("input shape",input.shape)
        x=input
        for index,layer in enumerate(self.model.named_children()):
            #print(index)
            #print(layer)
            x=layer(x)
            if (index == self.selected_layer):
                return x

    def get_single_feature(self):
        features=self.get_feature()
        print("features.shape",features.shape)
        feature=features[:,0,:,:]
        print(feature.shape)
        feature=feature.view(feature.shape[1],feature.shape[2])
        print(feature.shape)
        return features

    def save_feature_to_img(self):
        #to numpy
        features=self.get_single_feature()
        for i in range(features.shape[1]):
            feature = features[:, i, :, :]
            feature = feature.view(feature.shape[1], feature.shape[2])
            feature = feature.data.numpy()
            # use sigmod to [0,1]
            feature = 1.0 / (1 + np.exp(-1 * feature))
            # to [0,255]
            feature = np.round(feature * 255)
            print(feature[0])
            mkdir('./feature/' + str(self.selected_layer))
            cv2.imwrite('./feature/'+ str( self.selected_layer)+'/' +str(i)+'.jpg', feature)
if __name__=='__main__':
    # get class
    for  k in range(30):
        myClass=FeatureVisualization('/home/lqy/examples/TRP.PNG',k)
        print (myClass.pretrained_model)
        myClass.save_feature_to_img()